**извлечение признаков из текста на естественном языке**

классификатор текстов

частотный анализ, понижаем значение признака для частых слов (IDF)

Евгений Борисов borisov.e@solarl.ru

## библиотеки

In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score

In [3]:
pd.options.display.max_colwidth = 200  

## тексты

In [4]:
# загружаем тексты
data = pd.read_pickle('../data/text/news.pkl')
print('записей:',len(data))

записей: 3196


In [5]:
data.sample(2)

,text,tag
3127,"Восток-Медиа (vostokmedia.com) Дальневосточный фестиваль художественных\nремесел ""Живая нить времен"" пройдет в Хабаровске. В нем примут участие\nмастера художественных промыслов и декоративно-прик...",culture
560,Названы возможные причины пожара с гибелью троих молодых людей в Витебском районе\n\nопубликовано: 3 декабря 2016 в 14:20\n\nобновлено: 5 декабря 2016 в 12:17\n\nTUT.BY\n\nТрагедия случилась в дер...,incident


##  CountVectorizer + TF-IDF

In [6]:
def preprocessor(text):
    tt = [ t for t in text.split() if t ]
    tt = [ t.lower()  for t in tt ] # приведение в lowercase
    tt = [ re.sub( r'https?://[\S]+', 'url', t)  for t in tt ]  # замена интернет ссылок
    tt = [ re.sub( r'[\w\./]+\.[a-z]+', 'url', t) for t in tt  ]  # замена интернет ссылок 
    tt = [ re.sub( r'<[^>]*>', '', t)  for t in tt ] # удаление html тагов
    tt = [ re.sub( r'\W', '', t)  for t in tt ] # удаление лишних символов (НЕ буква и НЕ цифра)
    tt = [ re.sub( r'\b\d+\b', 'digit', t ) for t in tt ] # замена цифр
    return ' '.join( [ t.strip() for t in tt if t ] )
    

In [7]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer

# понижаем значение признака для частых слов (IDF)
tf = TfidfVectorizer(preprocessor=preprocessor,use_idf=True)
tf.fit( data['text'] )

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2',
        preprocessor=<function preprocessor at 0x7f60463ecbf8>,
        smooth_idf=True, stop_words=None, strip_accents=None,
        sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, use_idf=True, vocabulary=None)

In [8]:
# размер словаря
len(tf.vocabulary_)

85058

## формируем датасеты

In [9]:
X = tf.transform( data['text'] ) # .todense()
X.shape

(3196, 85058)

In [10]:
labels = { t:i for i,t in enumerate(sorted(set(data['tag']))) }
labels

{'auto': 0,
 'culture': 1,
 'economics': 2,
 'health': 3,
 'incident': 4,
 'politics': 5,
 'realty': 6,
 'reclama': 7,
 'science': 8,
 'social': 9,
 'sport': 10,
 'tech': 11,
 'woman': 12}

In [11]:
y = data['tag'].map(labels).values
y

array([5, 1, 1, ..., 8, 5, 9])

---

In [12]:
from time import time
def get_seed(): t = time() ; return int(((t%1)/(t//1))*1e11)

In [13]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.9, random_state=get_seed() )
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((319, 85058), (319,), (2877, 85058), (2877,))

## обучаем

In [14]:
from sklearn.linear_model import SGDClassifier

clf = SGDClassifier(loss='hinge',max_iter=1000)
clf.fit(X_train,y_train)

SGDClassifier(alpha=0.0001, average=False, class_weight=None,
       early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
       l1_ratio=0.15, learning_rate='optimal', loss='hinge', max_iter=1000,
       n_iter=None, n_iter_no_change=5, n_jobs=None, penalty='l2',
       power_t=0.5, random_state=None, shuffle=True, tol=None,
       validation_fraction=0.1, verbose=0, warm_start=False)

## тестируем

In [15]:
o = clf.predict(X_train)

In [16]:
accuracy_score(y_train,o)

1.0

In [17]:
# print( classification_report(y_train,o) )

---

In [18]:
o = clf.predict(X_test)

In [19]:
accuracy_score(y_test,o)

0.7511296489398679

In [20]:
print( classification_report(y_test,o) )

              precision    recall  f1-score   support

           0       0.83      0.79      0.81       224
           1       0.65      0.77      0.70       323
           2       0.86      0.48      0.62       256
           3       1.00      0.18      0.30        85
           4       0.72      0.95      0.82       389
           5       0.69      0.91      0.78       538
           6       0.80      0.14      0.24        56
           7       0.72      0.29      0.41        45
           8       0.88      0.96      0.91       205
           9       0.69      0.19      0.29       129
          10       0.95      0.92      0.93       339
          11       0.73      0.62      0.67       260
          12       0.37      0.71      0.49        28

   micro avg       0.75      0.75      0.75      2877
   macro avg       0.76      0.61      0.61      2877
weighted avg       0.77      0.75      0.73      2877



---

In [21]:
from matplotlib import pyplot as plt
import itertools

plt.figure(figsize=(10,9))

cm = confusion_matrix(y_test,o)
plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion matrix')
plt.colorbar()

classes = sorted(labels.keys())

tick_marks = np.arange(len(classes))
plt.xticks(tick_marks, classes, rotation=45)
plt.yticks(tick_marks, classes)

thresh = cm.max() / 2.
for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
    plt.text(j, i, format(cm[i, j], 'd'),
             horizontalalignment="center",
             color="white" if cm[i, j] > thresh else "black")

plt.tight_layout()
plt.ylabel('True label')
plt.xlabel('Predicted label')

plt.show()

<Figure size 1000x900 with 2 Axes>

---

In [22]:
o = clf.predict(X)

In [23]:
labels_inv = { labels[k]:k for k in labels}
# labels_inv

In [24]:
i = np.random.randint(len(data))
print('tag:',data.iloc[i,1])
print('predict:',labels_inv[o[i]])
print('- - - - - - - - - - - - - - - - - - \n')
print(data.iloc[i,0])


tag: auto
predict: auto
- - - - - - - - - - - - - - - - - - 

В интернете впервые опубликованы полноразмерные компьютерные изображения внедорожника Marussia F1, разрабатывавшегося компанией Marussia Motors.

Автомобиль Marussia F1 разрабатывался компанией в течение 2009-2010 годов по техническому заданию, предполагавшему создание внедорожника средних размеров розничной стоимостью около 1,7 миллиона рублей.

Автором внешнего вида Marussia F1 стал молодой дизайнер Иван Борисов, который затем ушел в Научный автомоторный институт (НАМИ). По каким-то причинам руководство Marussia Motors предпочло этому варианту более эксцентричную модель Marussia F2.

Характерно, что в голосовании на сайте drom.ru большинство участников - 71 процент - заявили, что дизайн Marussia F1 им не нравится. И всего лишь 21 процент голосов принадлежит тем, у кого фотографии вызвали положительный отклик.

Напомним, что компания Marussia Motors была создана в 2008 году. Инвесторы проекта заявляли о намерении наладить м

---

In [25]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# le.fit(data['tag'])
# print( list(le.classes_) )
# y = le.transform(data['tag']) 

In [26]:
# from sklearn.pipeline import Pipeline
# text_clf = Pipeline([
#                 ('tfidf', TfidfVectorizer()),
#                 ('clf', SGDClassifier(loss='hinge')),
#                 ])
# text_clf.fit()

In [27]:
# from sklearn.model_selection import GridSearchCV

---

In [28]:
# Sebastian Raschka   Python Machine Learning  - Packt Publishing Ltd, 2015